Nome: Rebeca Andrade Medeiros  
Matrícula: 11500782  

## Exercício - Análise de Sentimentos

A biblioteca sklearn será utilizada para a classificação com Naive Bayes e o cálculo de acurácia.

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score as a

In [2]:
df = pd.read_csv("B2W-Reviews01.csv")

<ipython-input-2-fd4adb6a3c4d>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("B2W-Reviews01.csv")


In [3]:
df.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


In [4]:
# Valores únicos da coluna 'recommend_to_a_friend'
sentimento = df['recommend_to_a_friend']
sentimento.unique()

array(['Yes', 'No', nan], dtype=object)

In [5]:
#review_text = df[['review_text', 'recommend_to_a_friend']]
review_text = df['review_text']

In [6]:
review_text = df['review_text'].str.lower()

In [7]:
review_text[0]

'estou contente com a compra entrega rápida o único problema com as americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.'

In [8]:
review_text = review_text.dropna()

In [9]:
sentimento = review_text.str.lower()
sentimento = sentimento.dropna()

#### Adição do prefixo “NAO_”

In [10]:
prefixo = 'NAO_'

In [11]:
def add_prefixo(review_text, prefixo):
    regex = r'(não|nem|nunca|jamais|tampouco)'
    add = re.sub(regex, rf'{prefixo}\1', review_text)
    return add

In [12]:
teste = 'a entrega foi no prazo, as americanas estão de parabéns. a smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. recomendo.'

In [13]:
print(add_prefixo(teste, prefixo))

a entrega foi no prazo, as americanas estão de parabéns. a smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, NAO_não trava, sem falar da imagem que é de surpreender. recomendo.


### Classificação sem prefixo

In [14]:
# Cópias dos reviews sem adicão do prefixo
review_sem_pref = review_text

In [15]:
review_sem_pref.head()

0    estou contente com a compra entrega rápida o ú...
1    por apenas r$1994.20,eu consegui comprar esse ...
2    supera em agilidade e praticidade outras panel...
3    meu filho amou! parece de verdade com tantos d...
4    a entrega foi no prazo, as americanas estão de...
Name: review_text, dtype: object

In [16]:
sem_pref = review_sem_pref.to_numpy()

In [17]:
sem_pref[0:3]

array(['estou contente com a compra entrega rápida o único problema com as americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.',
       'por apenas r$1994.20,eu consegui comprar esse lindo copo de acrílico.',
       'supera em agilidade e praticidade outras panelas elétricas.  costumo usar outra panela para cozimento de arroz (japonesa), mas leva muito tempo,  +/- 50 minutos.  nessa panela  é muito mais rápido, exatamente 6 minutos.    eu recomendo.'],
      dtype=object)

In [18]:
# Converter a entrada para array
sem_pref_vect = sem_pref
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(sem_pref_vect)

In [19]:
spref_array = tfidf.toarray()

In [20]:
spref_array.shape

(129098, 50336)

In [ ]:
# Separar dados entre teste e treino
spref_train, spref_teste = train_test_split(spref_array, test_size=0.2)

In [ ]:
# Classificador
class_bernouli = naive_bayes.BernoulliNB()

In [ ]:
class_bernouli.fit(spref_train)
pred = class_bernouli.predict(spref_teste)

In [ ]:
# Acurácia
print("Acurácia dos dados sem prefixo: ", a())